<a href="https://colab.research.google.com/github/Anupam-chand/NovaSystem/blob/main/Multi_Doc_QnA_with_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Installing Required Libraries
%pip install python-docx
%pip install python-pptx
%pip install PyPDF2
%pip install langchain
!pip install pandas
!pip install openpyxl
!pip install -q 'arize[AutoEmbeddings]'
!pip install -q "openai<1.0.0"
!pip install -q langchain_core
%pip install langchain_community
%pip install langchain_google_genai
%pip install langchain_text_splitters
%pip install sentence-transformers
%pip install faiss-cpu
%pip install cohere

  Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)
  Using cached langchain-0.2.9-py3-none-any.whl (987 kB)
  Using cached langchain_core-0.2.21-py3-none-any.whl (372 kB)
  Using cached langchain_text_splitters-0.2.2-py3-none-any.whl (25 kB)
  Using cached langsmith-0.1.90-py3-none-any.whl (134 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.2/135.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.5/178.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.5 MB/s eta 0:00:00


In [2]:
# necessary Imports
import os
import pandas as pd
from docx import Document
from PyPDF2 import PdfReader
from pptx import Presentation
from langchain_community.llms import Cohere
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import AIMessage, HumanMessage
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts  import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder

#Data Loading
I have taken Three different types of data.
 Which are :- PDF, PPT, DOCS & xlsx file.

In [4]:
pdf_file = open('/content/stories.pdf','rb')
ppt_file = Presentation("/content/Study of various Open Source and Commercial Database Management Systems (1).pptx")
doc_file = Document('/content/ANPR_Research_paper_merged[1][1][1].docx')
xlsx_file = pd.read_excel('/content/School-Report-Card-And-Mark-Sheet-Template.xlsx')

#Data Extraction

PDF :- Pdf data is extracted using PyPDF2 and all text is stored in a string.
PPT :- PPT data is extracted using python-pptx module and all text is stored in a string.
DOCS :- Docs data is extracted using python-docs module and all text is stored in a string.
After Extracting all data seperately, I have combined all text in a single string for further text processing.

In [5]:
# extracting pdf data
pdf_text = ""
pdf_reader = PdfReader(pdf_file)
for page in pdf_reader.pages:
    pdf_text += page.extract_text()

# extracting ppt data
ppt_text = ""
for slide in ppt_file.slides:
    for shape in slide.shapes:
        if hasattr(shape, "text"):
            ppt_text += shape.text + '\n'

# extracting doc data
doc_text = ""
for paragraph in doc_file.paragraphs:
    doc_text += paragraph.text + '\n'

# extracting xlsx data
xlsx_text = ""
for index, row in xlsx_file.iterrows():
    xlsx_text += str(row) + '\n'

In [6]:
# merging all the text

all_text = pdf_text + '\n' + ppt_text + '\n' + doc_text + '\n' + xlsx_text
len(all_text)

265930

#Chunking

In this step I am creating the chunks of data, for this step I am using Recursive Character Splitter which break large Documents into smaller chunks. This is useful both for indexing data and for passing it in to a model, since large chunks are harder to search over and won’t fit in a model’s finite context window.

In [7]:
# splitting the text into chunks for embeddings creation

text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 1000,
        chunk_overlap = 200, # This is helpul to handle the data loss while chunking.
        length_function = len,
        separators=['\n', '\n\n', ' ', '']
    )

chunks = text_splitter.split_text(text = all_text)

In [8]:
len(chunks)

347

#Embeddings Creation

Embeddings creation is a crucial preprocessing step in the development of document-based Question and Answering (Q&A) systems. This process involves converting textual data from documents and questions into dense, high-dimensional vectors known as embeddings. These embeddings are designed to capture the semantic meaning of words, sentences, or even entire documents, enabling the Q&A system to understand and process natural language more effectively.

In [9]:
# Initializing embeddings model

embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

#Indexing

Indexing data using Facebook AI Similarity Search (FAISS) is a pivotal step in developing efficient and scalable document-based Question and Answering (Q&A) systems. FAISS is a library that facilitates the efficient search for similarities in large datasets, especially useful for tasks involving high-dimensional vectors like text embeddings. When applied to document-based Q&A, FAISS indexes the embeddings of document chunks (e.g., paragraphs, sentences) to optimize the retrieval process.

In [10]:
# Indexing the data using FAISS
vectorstore = FAISS.from_texts(chunks, embedding = embeddings)

#Retriever

In the development of document-based Question and Answering (Q&A) systems, creating a retriever is a crucial step that directly impacts the system's ability to find relevant information efficiently. The retriever utilizes the pre-indexed embeddings of document chunks, searching through them to find the most relevant pieces of content in response to a user query. This process involves setting up a retrieval mechanism that leverages similarity search to identify the best matches for the query embeddings within the indexed data.

In [11]:
# creating retriever
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [12]:
retrieved_docs = retriever.invoke("How tortoise wins the race in THE HARE AND THE TORTOISE story?")

In [13]:
len(retrieved_docs)

6

In [14]:
print(retrieved_docs[0].page_content)

but I have caught you with those were destroying my crops, and youmust suffer with the company in which you are found.”
People are judged by the company they keep.THE BOY WHO CRIED 'WOLF'
100 Moral Stories                                                  5                                      www.islamicoccasions. comA tortoise one day met a hare who made fun of her. 
“My, my, you move so slowly, you will never get far!”
The tortoises, upset by the hare’s manner, said, 
“Let’s have a race and see who is faster.”
The hare laughed and said, “You must be joking! But all right, we’ll
see who reaches the other side of the hill first.” Off he ran, leavingthe tortoise far behind.
After a while, the hare stopped to wait for the tortoise to come long. 
He waited and waited till he felt sleepy. “I might as well take a nap,” he thought. “Even if she catches up with me, I can easily win therace.” So he lay down under a shady tree and closed his eyes.


#LLM Models

Large Language Models (LLMs) are advanced artificial intelligence systems designed to understand, generate, and interact with human language in a way that mimics human-like understanding. These models are trained on vast amounts of text data, allowing them to grasp the nuances of language, including grammar, context, and even cultural references. The capabilities of LLMs extend beyond simple text generation; they can perform a variety of tasks such as translation, summarization, question answering, and even code generation.
One of the key technologies behind LLMs is the Transformer architecture, which enables the model to pay attention to different parts of the input text differently, thereby understanding the context and relationships between words and phrases more effectively. This architecture has led to significant improvements in natural language processing tasks and is the foundation of many state-of-the-art LLMs.

#Cohere LLM

In [15]:
import os
os.environ["COHERE_API_KEY"] = "15eAupVjoeYXxpofPqjw3C0utRWwGZ9QLgSzYQig"

In [16]:
cohere_llm = Cohere(model="text-davinci-003", temperature=0.1, cohere_api_key="15eAupVjoeYXxpofPqjw3C0utRWwGZ9QLgSzYQig")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Cohere` was deprecated in LangChain 0.1.14 and will be removed in 0.3.0. An updated version of the class exists in the langchain-cohere package and should be used instead. To use it run `pip install -U langchain-cohere` and import as `from langchain_cohere import Cohere`.
  warn_deprecated(


In [17]:
prompt_template = """Answer the question as precise as possible using the provided context. If the answer is
                not contained in the context, say "answer not available in context" \n\n
                Context: \n {context}?\n
                Question: \n {question} \n
                Answer:"""

prompt = PromptTemplate.from_template(template=prompt_template)

In [18]:
# function to create a single string of relevant documents given by Faiss.
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [19]:
# RAG Chain

def generate_answer(question):
    cohere_llm = Cohere(model="command", temperature=0.1, cohere_api_key = os.getenv('COHERE_API_KEY'))

    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | cohere_llm
        | StrOutputParser()
    )

    return rag_chain.invoke(question)

#Results

In [27]:
ans = generate_answer("Summarize the grading system for 100 marks and 50 marks")
print(ans)

 The grading system for 100 marks is as follows:
1st Div: 90-100 Marks = A+
2nd Div: 80-89 Marks = A
3rd Div: 70-79 Marks = B+
4th Div: 60-69 Marks = B
5th Div: 50-59 Marks = C+
6th Div: 0-49 Marks = O

The grading system for 50 marks is as follows:
1st Div: 45-50 Marks = A+
2nd Div: 40-44 Marks = A
3rd Div: 35-39 Marks = B+
4th Div: 30-34 Marks = B
5th Div: 0-29 Marks = X


In [ ]:
ans = generate_answer("Why the hare and tortoise decide to race?")
print(ans)

 To see who is faster. The hare made fun of the tortoise one day and said, “You must be joking! But all right, we’ll
see who reaches the other side of the hill first.”


In [ ]:
ans = generate_answer("What is PostgreSQL?")
print(ans)

 PostgreSQL is an open-source object-relational database management system that provides robust and reliable performance. It supports a wide range of data types and offers advanced features for handling complex data structures. PostgreSQL supports SQL (Structured Query Language) for querying and manipulating data and provides support for procedural languages such as PL/PGSQL, PL/Python, PL/Perl, and PL/Java. It also supports various commands for managing databases, tables, indexes, and users. 

PostgreSQL is an example of one of many open-source Database Management Systems (DBMS) that can be utilized by companies. 


In [ ]:
ans = generate_answer("provide brief summary about ANPR research paper?")
print(ans)

 The research paper discusses the benefits of ANPR (Automatic Number Plate Recognition) systems in identifying license plates from vehicle images to ensure proper traffic management and reduce the cause of road accidents. The paper also discusses the limitations of ANPR systems, including weather conditions, poor image quality, angle and distance, obstructions, non-standardized number plates, reflections and glare, and vehicle design. The paper concludes by discussing the importance of designing ANPR systems with privacy safeguards and addressing the challenges of processing large volumes of data in real-time. 


In [ ]:
ans = generate_answer("Who is a president of india?")
print(ans)

 answer not available in context
